In [1]:
USE_GPU = True
GPU_ID = "5" # the gpu to use for this job
if USE_GPU:
    import os
    os.environ["CUDA_VISIBLE_DEVICES"] = GPU_ID
    # sanity check that only one gpu is available
#     from tensorflow.python.client import device_lib
#     print(device_lib.list_local_devices())

In [2]:
import tensorflow as tf
# Suppress the annoying tensorflow 1.x deprecation warnings; these make console output
# impossible to parse.
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [3]:
# initialize logger to view logging output from segmentation tasks
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [4]:
from google.protobuf import text_format
from ffn.inference import inference
from ffn.inference import inference_pb2

In [5]:
config = '''image {
  hdf5: "data/test/507727402/507727402_raw.h5:raw"
}
image_mean: 78
image_stddev: 20
checkpoint_interval: 1800
seed_policy: "PolicyPeaks"
model_checkpoint_path: "training-logs/lr0.001depth9/model.ckpt-1688434"
model_name: "fftracer.training.models.model.FFNTracerModel"
model_args: "{\\"depth\\": 9, \\"fov_size\\": [49, 49, 1], \\"deltas\\": [8, 8, 0]}"
segmentation_output_dir: "results/mozak"
inference_options {
  init_activation: 0.95
  pad_value: 0.05
  move_threshold: 0.1
  min_boundary_dist { x: 1 y: 1 z: 0}
  segment_threshold: 0.01
  min_segment_size: 1000
}'''
req = inference_pb2.InferenceRequest()
_ = text_format.Parse(config, req)

In [6]:
runner = inference.Runner()
runner.start(req)
canvas, alignment = runner.make_canvas((0, 0, 0), (1, 9429, 7601)) # sizes in (z, y, x)

DEBUG:root:Received request:
image_mean: 78.0
image_stddev: 20.0
model_name: "fftracer.training.models.model.FFNTracerModel"
model_args: "{\"depth\": 9, \"fov_size\": [49, 49, 1], \"deltas\": [8, 8, 0]}"
model_checkpoint_path: "training-logs/lr0.001depth9/model.ckpt-1688434"
inference_options {
  init_activation: 0.949999988079071
  pad_value: 0.05000000074505806
  move_threshold: 0.10000000149011612
  min_boundary_dist {
    x: 1
    y: 1
    z: 0
  }
  segment_threshold: 0.009999999776482582
  min_segment_size: 1000
}
segmentation_output_dir: "results/mozak"
checkpoint_interval: 1800
seed_policy: "PolicyPeaks"
image {
  hdf5: "data/test/507727402/507727402_raw.h5:raw"
}

/homes/gws/jpgard/ffn/ffn/inference/storage.py:62: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READON

In [ ]:
### EXPERIMENTAL: show the input image data to verify it is valid
import skimage
from matplotlib import pyplot as plt
skimage.io.imshow(canvas.image[0,:,:])
plt.show()
### END

In [ ]:
# Create a single segment, starting from the specified origin point.
canvas.segment_at(
    (0, 4521, 3817),  # seed starting point in (z, y, x)
    dynamic_image=inference.DynamicImage(),
    vis_update_every=50,
    vis_fixed_z=True
)

In [ ]:
# read the results
import numpy as np
seg_prob = np.load("results/mozak/0/0/seg-0_0_0.prob")['qprob']
print("shape: {}".format(seg_prob.shape))
seg_prob